Bu notebook'ta farklı öznitelik seçim yöntemleriyle elde edilen veri setleri üzerinde model eğitimi yapılacak ve performansları karşılaştırılacaktır:

1. **Baz Model** - Tüm özniteliklerle
2. **Filter Method Modeli** - SelectKBest ile seçilen özniteliklerle
3. **RFE Method Modeli** - RFE ile seçilen özniteliklerle
4. **Embedded Method Modeli** - Random Forest importance ile seçilen özniteliklerle

Her model için **5-Fold Cross-Validation** ile değerlendirme yapılacaktır.

### 1. Kutuphanelerin Import Edilmesi

In [ ]:
import pandas as pd
import numpy as np  
import plotly.express as px
import pickle
import time
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, classification_report,
                             confusion_matrix, roc_curve)


### 2. İşlenmiş Veriyi ve Secilmis Oznitelikleri Yükleme 

In [ ]:
# İşlenmiş veriyi yükle
X_train = pd.read_csv('../data/processed/X_train.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')
y_train = pd.read_csv('../data/processed/y_train.csv').values.ravel()
y_test = pd.read_csv('../data/processed/y_test.csv').values.ravel()

# Seçilmiş öznitelikleri yükle
with open('../data/processed/selected_features.pkl', 'rb') as f:
    selected_features = pickle.load(f)

print(f"Veri Seti Boyutları:")
print(f"  X_train: {X_train.shape}")
print(f"  X_test: {X_test.shape}")
print(f"\nSeçilmiş Öznitelik Sayıları:")
print(f"  Tüm Öznitelikler: {len(selected_features['all_features'])}")
print(f"  Filter Method: {len(selected_features['filter'])}")
print(f"  RFE Method: {len(selected_features['rfe'])}")
print(f"  Embedded Method: {len(selected_features['embedded'])}")

### 3. Veri Setlerinin Hazırlanması

In [ ]:
# 4 farklı veri seti oluştur
datasets = {
    'Base (All Features)': {
        'X_train': X_train,
        'X_test': X_test,
        'n_features': X_train.shape[1]
    },
    'Filter Method': {
        'X_train': X_train[selected_features['filter']],
        'X_test': X_test[selected_features['filter']],
        'n_features': len(selected_features['filter'])
    },
    'RFE Method': {
        'X_train': X_train[selected_features['rfe']],
        'X_test': X_test[selected_features['rfe']],
        'n_features': len(selected_features['rfe'])
    },
    'Embedded Method': {
        'X_train': X_train[selected_features['embedded']],
        'X_test': X_test[selected_features['embedded']],
        'n_features': len(selected_features['embedded'])
    }
}

print("Veri Setleri Hazırlandı:")
print("="*60)
for name, data in datasets.items():
    print(f"{name:25s} -> {data['n_features']:2d} öznitelik")

### 4. Model Tanımlama ve Değerlendirme Fonksiyonu

In [ ]:
# Finansal risk tespiti için Random Forest kullanıyorum
# class_weight='balanced' ile dengesiz veriye karşı uyguluyorum

from sklearn.ensemble import RandomForestClassifier

def evaluate_model(X_train, y_train, X_test, y_test, model_name):
    """
    Model eğitimi ve 5-fold cross-validation ile değerlendirme
    """
    # Model tanımla
    model = RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        class_weight='balanced',  # Dengesiz veri için
        random_state=42,
        n_jobs=-1
    )
    
    # 5-Fold Cross Validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    
    start_time = time.time()
    cv_results = cross_validate(
        model, X_train, y_train,
        cv=cv, scoring=scoring, return_train_score=False
    )
    cv_time = time.time() - start_time
    
    # Model eğit (tüm train veriyle)
    model.fit(X_train, y_train)
    
    # Test seti tahminleri
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Sonuçları dictionary'de topla
    results = {
        'model_name': model_name,
        'cv_accuracy_mean': cv_results['test_accuracy'].mean(),
        'cv_accuracy_std': cv_results['test_accuracy'].std(),
        'cv_precision_mean': cv_results['test_precision'].mean(),
        'cv_precision_std': cv_results['test_precision'].std(),
        'cv_recall_mean': cv_results['test_recall'].mean(),
        'cv_recall_std': cv_results['test_recall'].std(),
        'cv_f1_mean': cv_results['test_f1'].mean(),
        'cv_f1_std': cv_results['test_f1'].std(),
        'cv_roc_auc_mean': cv_results['test_roc_auc'].mean(),
        'cv_roc_auc_std': cv_results['test_roc_auc'].std(),
        'test_accuracy': accuracy_score(y_test, y_pred),
        'test_precision': precision_score(y_test, y_pred),
        'test_recall': recall_score(y_test, y_pred),
        'test_f1': f1_score(y_test, y_pred),
        'test_roc_auc': roc_auc_score(y_test, y_pred_proba),
        'training_time': cv_time,
        'model': model,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    return results
print("fonksiyon tanımlandı.")

### 5. Modellerin Eğitimi ve Değerlendirilmesi

In [ ]:
all_results = []

print("Modeller eğitiliyor...")
print("="*70)

for dataset_name, dataset in datasets.items():
    print(f"\n{dataset_name} ({dataset['n_features']} öznitelik)...")
    
    results = evaluate_model(
        dataset['X_train'], y_train,
        dataset['X_test'], y_test,
        model_name=dataset_name
    )
    
    all_results.append(results)
    
    print(f"  ✓ CV ROC-AUC: {results['cv_roc_auc_mean']:.4f} (±{results['cv_roc_auc_std']:.4f})")
    print(f"  ✓ Test ROC-AUC: {results['test_roc_auc']:.4f}")
    print(f"  ✓ Test Recall: {results['test_recall']:.4f}")
    print(f"  ✓ Eğitim Süresi: {results['training_time']:.2f}s")

print("\n" + "="*70)
print("Tüm modeller eğitildi!")

### 6. Sonuclarin Karsilastirilmasi

In [ ]:
# Sonuçları DataFrame'e çevirir
comparison_df = pd.DataFrame([{
    'Model': r['model_name'],
    'Features': datasets[r['model_name']]['n_features'],
    'CV Accuracy': f"{r['cv_accuracy_mean']:.4f} ± {r['cv_accuracy_std']:.4f}",
    'CV Precision': f"{r['cv_precision_mean']:.4f} ± {r['cv_precision_std']:.4f}",
    'CV Recall': f"{r['cv_recall_mean']:.4f} ± {r['cv_recall_std']:.4f}",
    'CV F1': f"{r['cv_f1_mean']:.4f} ± {r['cv_f1_std']:.4f}",
    'CV ROC-AUC': f"{r['cv_roc_auc_mean']:.4f} ± {r['cv_roc_auc_std']:.4f}",
    'Test Accuracy': f"{r['test_accuracy']:.4f}",
    'Test Recall': f"{r['test_recall']:.4f}",
    'Test ROC-AUC': f"{r['test_roc_auc']:.4f}",
    'Training Time (s)': f"{r['training_time']:.2f}"
} for r in all_results])

print("Model Karşılaştırma Tablosu:")
print("="*100)
print(comparison_df.to_string(index=False))
print("="*100)

#### 6.1 Karsilastirma Metriklerinin Gorsellestirilmesi

In [ ]:
# CV metrikleri karşılaştırması
metrics = ['cv_accuracy_mean', 'cv_precision_mean', 'cv_recall_mean', 'cv_f1_mean', 'cv_roc_auc_mean']
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=metric_names + ['Training Time'],
    specs=[[{}, {}, {}], [{}, {}, {}]]
)

positions = [(1,1), (1,2), (1,3), (2,1), (2,2)]

for (row, col), metric, name in zip(positions, metrics, metric_names):
    for result in all_results:
        fig.add_trace(
            go.Bar(
                name=result['model_name'],
                x=[result['model_name']],
                y=[result[metric]],
                error_y=dict(type='data', array=[result[metric.replace('_mean', '_std')]]),
                showlegend=(row==1 and col==1)
            ),
            row=row, col=col
        )

# Training time ekle
for result in all_results:
    fig.add_trace(
        go.Bar(
            name=result['model_name'],
            x=[result['model_name']],
            y=[result['training_time']],
            showlegend=False
        ),
        row=2, col=3
    )

fig.update_layout(
    title_text="Model Performans Karşılaştırması (5-Fold Cross-Validation)",
    height=600,
    showlegend=True
)

fig.update_yaxes(title_text="Score", row=1, col=1)
fig.update_yaxes(title_text="Score", row=1, col=2)
fig.update_yaxes(title_text="Score", row=1, col=3)
fig.update_yaxes(title_text="Score", row=2, col=1)
fig.update_yaxes(title_text="Score", row=2, col=2)
fig.update_yaxes(title_text="Seconds", row=2, col=3)

fig.show()

#### 6.2 ROC-AUC ve Recall odaklı karşılaştırma

In [ ]:
# ROC-AUC ve Recall odaklı karşılaştırma (Finansal risk için önemli)
fig = go.Figure()

model_names = [r['model_name'] for r in all_results]
roc_auc_scores = [r['cv_roc_auc_mean'] for r in all_results]
recall_scores = [r['cv_recall_mean'] for r in all_results]

fig.add_trace(go.Bar(
    name='ROC-AUC',
    x=model_names,
    y=roc_auc_scores,
    marker_color='steelblue'
))

fig.add_trace(go.Bar(
    name='Recall',
    x=model_names,
    y=recall_scores,
    marker_color='orange'
))

fig.update_layout(
    title="Finansal Risk İçin Kritik Metrikler: ROC-AUC ve Recall",
    xaxis_title="Model",
    yaxis_title="Score",
    barmode='group',
    height=500
)

fig.show()

### 7. Modellerin Kaydedilmesi

In [ ]:
import os
os.makedirs('../models', exist_ok=True)

# Her model kaydediliyor.
for result in all_results:
    model_name = result['model_name']
    model = result['model']
    
    # Dosya adlari duzenleniyor
    filename = model_name.replace(' ', '_').replace('(', '').replace(')', '') + '.pkl'
    filepath = os.path.join('../models', filename)
    
    # Modeli kaydet
    with open(filepath, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"✓ {model_name} modeli kaydedildi: {filename}")

print("\n" + "="*70)
print("Tüm modeller başarıyla kaydedildi!")